In [9]:
# Standard Python Imports

import numpy as np
import pandas as pd
import csv
#from numba import njit
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.formula.api as sm
import math
from scipy.stats import norm
from random import choices
from numpy import matlib
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from scipy.stats import mode
from scipy.sparse import identity
from scipy.sparse.linalg import inv as spinv
from scipy.sparse.linalg import spsolve as spsolve
from numpy.linalg import eig

inv, ax, randint = np.linalg.inv, np.newaxis, np.random.randint
repmat, rand = np.matlib.repmat, np.random.rand

## Read in data

In [5]:
datapath = "/Users/nadialucas/Documents/fabfour/pset1/va-data.csv"
df = pd.read_csv(datapath)
df.head()



,year,id_school,id_grade,id_teacher,id_class,id_student,id_cohort,m_education,hh_income,sd_indv,score_std,lag_score_std
0,2000,1,3,1,3,6,6,8,13566.4960,0.090079,0.370097,NaN
1,2000,1,3,1,3,3906,6,12,4036.5371,0.090079,-2.011367,NaN
2,2000,1,3,1,3,4556,6,21,15218.5520,0.090079,0.732662,NaN
3,2000,1,3,1,3,5206,6,18,11004.5960,0.090079,-0.251898,NaN
4,2000,1,3,1,3,10406,6,11,7115.0620,0.090079,-1.278978,NaN


## Estimate teacher VA from student scores (Appendix A)

Drift is accounted for by permitting coefficients on score data to vary non-parametrically according to the distance between observed score and forecast year

### Step 1: Residualization of test scores

Residualize student scores with respect to controls by running OLS with teacher fixed effects

In [15]:
# teacher on student scores OLS
# what should controls, X_it be?
# (i) cubic polynomial in prior-year scores, interacted with student grade level
# (ii) class and school-year means of all other individual covariates
# (iii) class size and class type indicators
# (iv) grade and year dummies

# I haven't done this yet, this is wrong
teacher_FE = sm.ols(formula = 'score_std ~ m_education + hh_income + m_education + C(id_teacher)', data = df).fit()

# then residualize to get A_it

### Step 2: Estimation of variance component

1. Estimate individual level variance of residual test scores

sigma_eps = MSE * (N-1)/(N-K-C+1)

where MSE is variance of within-classroom deviations of A_it, N is total number of students, C is total number of classrooms, and K is number of control variables in the X_it vector

2. Then construct precision-weighted averages of classroom-average scores within a teacher-year

weight_ct = 1/(sigma_theta + (sigma_eps/n_ct))

where sigma_theta is estimate of class-level variance and n_ct is num student sin classroom

3. Now estimate sigma_A0 which is covariance of test scores of adjacent classrooms in each teacher-year cell (weighting each pair by sum of students taught)

4. Then estimate sigma_As which is covariance between mean scores across years within teachers

### Step 3: Construction of VA estimates

1. Construct bar(A_j_-t) = vector of teacher-year-mean scores used to predict teacher j's VA in year t. N_jt is the length of this vector

2. Construct BLP of teacher quality in year t as 

mu_jt = (inv(Sigma_Ajt)\*gamma_jt)'\*bar(A_j_-t)

where gamma_jt is N_jt x 1, Sigma_Ajt is N_jt x N_jt - see appendix for what each element is

and we are done